## dekoratorių deklaracija

In [58]:
vaisiai = ["braškės", "bananai", "apelsinas"]
ilgiai = map(len, vaisiai)
print(list(ilgiai))



[7, 7, 9]


In [59]:
from typing import Callable

def pasisveikinimas(funkcija: Callable) -> None:
    def wrapper(vardas: str):
        print(f"sveikas, {vardas}")
        funkcija(vardas)
    return wrapper

@pasisveikinimas
def atsisveikinimas(vardas: str) -> None:
    print(f"viso gero, {vardas}")

atsisveikinimas("zmogau")

sveikas, zmogau
viso gero, zmogau


In [74]:
from typing import Callable, Any
from functools import wraps

skaiciai = [3, 7, 5, 4, 10]


def tikrinti_teigiamus(funkcija: Callable[..., int|float]) -> Callable[..., int|float]:
    """
    Dekoratorius, kuris patikrina visus argumentus, ar jie yra int ar float,
    ir ar jų reukšmės yra teigiamos. Jeigu bent vienas argumentas neatitinka
    reikalavimų, iššaukiama klaida

    Argumentai:
        funkcija (callable): funkcija, kurią reikia dekoruoti
    """
    @wraps(funkcija)
    def wrapper(*args, **kwargs) -> Any:
        """ 
        Dekoratoriaus wrapperis, kuris išmeta klaidą, jeigu paduotos funkcijos
        argai ir kwargų reikšmės nėra skaičiai arba yra neigiami.
        """
        if not all(type(arg) in (int, float) for arg in args) \
                or not all(type(kwarg) in (int, float) for kwarg in kwargs.values()):
            raise ValueError("Argumentai gali būti tik skaičiai")
        elif not all(arg > 0 for arg in args) \
                or not all(kwarg > 0 for kwarg in kwargs.values()):
            raise ValueError('Skaičiai turi būti visi teigiami')
        else:
            return funkcija(*args, **kwargs)
    return wrapper


@tikrinti_teigiamus
def sudauginti_ir_prideti_kwargus(*args, **kwargs) -> int|float:
    """
    Funkcija, kuri sudaugina argus, ir prie jos prideda kwargus
    """
    multiplied = 1
    for number in args:
        multiplied *= number
    if kwargs:
        for kwarg in kwargs.values():
            multiplied += kwarg
    return multiplied

print(sudauginti_ir_prideti_kwargus(*skaiciai, **{'bulvės': 69, 'kečupo': 11}))

4280


In [77]:
print(sudauginti_ir_prideti_kwargus.__doc__)
print(tikrinti_teigiamus.__doc__)


    Funkcija, kuri sudaugina argus, ir prie jos prideda kwargus
    

    Dekoratorius, kuris patikrina visus argumentus, ar jie yra int ar float,
    ir ar jų reukšmės yra teigiamos. Jeigu bent vienas argumentas neatitinka
    reikalavimų, iššaukiama klaida

    Argumentai:
        funkcija (callable): funkcija, kurią reikia dekoruoti
    


## klasės dekoratoriai

In [62]:
class Zmogus:
    gyvas = True

    def __init__(self, vardas:str, pavarde:str) -> None:
        self.vardas = vardas
        self.pavarde = pavarde

    @property
    def pilnas_vardas(self) -> str:
        return f"{self.vardas} {self.pavarde}"
    
    def __str__(self) -> str:
        return self.pilnas_vardas
    
    @staticmethod
    def eiti():
        print("einu")

    @classmethod
    @property
    def nemirtingas(cls):
        return cls.gyvas

petras = Zmogus("Petras", "Jonaitis")
print(petras)
petras.eiti()
Zmogus.eiti()
petras.gyvas = False
print(petras.nemirtingas, petras.gyvas)
print(Zmogus.nemirtingas)

Petras Jonaitis
einu
einu
True False
True


## dekoratoriai su argumentais

In [90]:
from functools import wraps
from typing import Callable, Any

def repeat(times: int):
    def repeat_decorator(func, times=times):
        @wraps(func)
        def wrapper(*args, times=times, **kwargs):
            results = []
            while times > 0:
                results.append(func(*args, **kwargs))
                times -= 1
            return results
        return wrapper
    return repeat_decorator

@repeat(5)
def print_message(message: str) -> None:
    print(message)

print_message("Labas")

Labas
Labas
Labas
Labas
Labas


[None, None, None, None, None]

In [186]:
from time import time
def meter(func):
    def wrapper(*args, **kwargs):
        start = time()
        result = func(*args, **kwargs)
        print(time() - start, func.__name__)
        return result
    return wrapper


In [197]:
from random import randint

@meter
@repeat(7)
def loterijos_bilietas():
    return randint(10, 99)

@repeat(5)
def laimingi_skaiciai():
    return randint(10, 99)

@meter
def tikrinti_bilietus(bilietu=1, loterija=laimingi_skaiciai()):
    bilietai = [loterijos_bilietas() for _ in range(bilietu)]
    print('laimingi skaiciai:', loterija)
    rezultatai = []
    for bilietas in bilietai:
        sutapo = 0
        for numeris in bilietas:
            if numeris in loterija:
                sutapo += 1
        rezultatai.append(sutapo)
        print('bilietas:', bilietas, 'sutapo:', sutapo)
    return rezultatai

print(tikrinti_bilietus(bilietu=3))

7.867813110351562e-06 loterijos_bilietas
1.049041748046875e-05 loterijos_bilietas
6.67572021484375e-06 loterijos_bilietas
laimingi skaiciai: [35, 45, 97, 73, 48]
bilietas: [67, 95, 73, 42, 21, 32, 35] sutapo: 2
bilietas: [34, 30, 89, 89, 87, 74, 82] sutapo: 0
bilietas: [51, 57, 98, 23, 41, 24, 15] sutapo: 0
0.0001533031463623047 tikrinti_bilietus
[2, 0, 0]
